In [20]:
import csv
import pandas as pd
import numpy as np

In [21]:
data = pd.read_csv('data.csv')

/home/alex/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6,10,22,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
data.head()

,raw_row_number,date,time,location,lat,lng,beat,subject_age,subject_race,subject_sex,...,contraband_weapons,search_conducted,search_basis,reason_for_stop,use_of_force_description,raw_subject_sdrace,raw_subject_resultofencounter,raw_subject_searchconducted,raw_subject_typeofsearch,raw_subject_resultofsearch
0,17213,NaN,01:33:00,E. 28th St. & Park BLVD,37.803084,-122.237247,NaN,NaN,asian/pacific islander,female,...,NaN,False,NaN,Traffic Violation,NaN,A,"Citation,",No,NaN,NaN
1,17214,NaN,14:48:00,1759 SEMINARY AV,37.767568,-122.196820,NaN,NaN,black,male,...,False,True,other,Probation/Parole,NaN,B,"FI Report,",Yes,"Prob./Parole,",NaN
2,1,2013-04-01,00:00:00,31st St And Mlk Jr Way,37.820599,-122.270734,NaN,NaN,white,male,...,False,True,other,Traffic Violation,NaN,W,"Warning,",Yes,"Prob./Parole,",NaN
3,2,2013-04-01,00:01:00,3000 Block Of San Pablo Ave,37.821246,-122.276488,NaN,NaN,black,male,...,NaN,False,NaN,Traffic Violation,NaN,B,"FI Report,",No,NaN,NaN
4,19,2013-04-01,10:41:00,12th St/Broadway,37.802937,-122.271738,NaN,NaN,hispanic,female,...,NaN,False,NaN,Probable Cause,NaN,H,"Citation,",No,NaN,NaN


In [23]:
set(data['subject_race'])

{'asian/pacific islander', 'black', 'hispanic', 'other', 'white'}

In [24]:
data.columns

Index(['raw_row_number', 'date', 'time', 'location', 'lat', 'lng', 'beat',
       'subject_age', 'subject_race', 'subject_sex', 'officer_assignment',
       'type', 'arrest_made', 'citation_issued', 'warning_issued', 'outcome',
       'contraband_found', 'contraband_drugs', 'contraband_weapons',
       'search_conducted', 'search_basis', 'reason_for_stop',
       'use_of_force_description', 'raw_subject_sdrace',
       'raw_subject_resultofencounter', 'raw_subject_searchconducted',
       'raw_subject_typeofsearch', 'raw_subject_resultofsearch'],
      dtype='object')

In [25]:
data['action'] = data['citation_issued'] | data['arrest_made']

In [26]:
inputs = ['subject_race', 'subject_sex', 'lat', 'lng', 'subject_age', 'reason_for_stop']
label = ['action']

In [27]:
data['subject_age'].median()

29.0

In [28]:
data['subject_age'] = data['subject_age'].fillna(data['subject_age'].median())

In [29]:
set(data['reason_for_stop'])

{'Consensual Encounter',
 'Consensual Encounter|Consensual Encounter|Consensual Encounter|Traffic Violation',
 'Consensual Encounter|Consensual Encounter|Probable Cause',
 'Consensual Encounter|Consensual Encounter|Probable Cause|Reasonable Suspicion',
 'Consensual Encounter|Consensual Encounter|Reasonable Suspicion|Reasonable Suspicion',
 'Consensual Encounter|Consensual Encounter|Traffic Violation',
 'Consensual Encounter|Probable Cause',
 'Consensual Encounter|Probable Cause|Consensual Encounter',
 'Consensual Encounter|Probable Cause|Consensual Encounter|Probable Cause',
 'Consensual Encounter|Probable Cause|Probable Cause|Probable Cause|Probable Cause',
 'Consensual Encounter|Probable Cause|Probation/Parole',
 'Consensual Encounter|Probation/Parole',
 'Consensual Encounter|Reasonable Suspicion',
 'Consensual Encounter|Reasonable Suspicion|Reasonable Suspicion',
 'Consensual Encounter|Traffic Violation',
 'Consensual Encounter|Traffic Violation|Traffic Violation',
 'Other-Consensua

In [30]:
data[inputs + label].head()

,subject_race,subject_sex,lat,lng,subject_age,reason_for_stop,action
0,asian/pacific islander,female,37.803084,-122.237247,29.0,Traffic Violation,True
1,black,male,37.767568,-122.196820,29.0,Probation/Parole,False
2,white,male,37.820599,-122.270734,29.0,Traffic Violation,False
3,black,male,37.821246,-122.276488,29.0,Traffic Violation,False
4,hispanic,female,37.802937,-122.271738,29.0,Probable Cause,True


In [31]:
races = list(set(data['subject_race']))
sexes = list(set(data['subject_sex']))
reasons = list(set(data['reason_for_stop']))

In [32]:
data['clean_lng'] = data['lng'] - data['lng'].mean() 
data['clean_lat'] = data['lat'] - data['lat'].mean()
data['clean_age'] = data['subject_age'] - data['subject_age'].mean()

In [33]:
data['race'] = data.apply(lambda row: races.index(row['subject_race']), axis=1)
data['sex']= data.apply(lambda row: sexes.index(row['subject_sex']), axis=1)
data['reason']= data.apply(lambda row: reasons.index(row['reason_for_stop']), axis=1).astype(int)

In [34]:
inputs = ['race', 'sex', 'clean_lat', 'clean_lng', 'clean_age']#, 'reason']
label = 'action'

## Linear Model

In [35]:
from sklearn.linear_model import LinearRegression

In [43]:
data_clean = data[inputs + [label]].dropna(axis = 0, subset= inputs + [label])

In [48]:
X = data_clean[inputs]
y = data_clean[label]
X.shape, y.shape

((133293, 5), (133293,))

In [52]:
reg = LinearRegression().fit(X, y)
np.mean((reg.predict(X) > .5) == y)


0.5351593857141785

In [53]:
from sklearn.linear_model import LogisticRegression

In [54]:
reg = LogisticRegression().fit(X, y)
np.mean((reg.predict(X) > .5) == y)


/home/alex/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.5354519742222023

In [55]:
from sklearn.neural_network import MLPClassifier
reg = MLPClassifier().fit(X, y)
np.mean((reg.predict(X) > .5) == y)


0.5532098459784085